In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
file = pd.read_csv('./comment-classification/ai_challenger_sentiment_analysis_trainingset_20180816/sentiment_analysis_trainingset.csv')

In [3]:
file.head(2)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2


In [4]:
file.isnull().sum()

id                                          0
content                                     0
location_traffic_convenience                0
location_distance_from_business_district    0
location_easy_to_find                       0
service_wait_time                           0
service_waiters_attitude                    0
service_parking_convenience                 0
service_serving_speed                       0
price_level                                 0
price_cost_effective                        0
price_discount                              0
environment_decoration                      0
environment_noise                           0
environment_space                           0
environment_cleaness                        0
dish_portion                                0
dish_taste                                  0
dish_look                                   0
dish_recommendation                         0
others_overall_experience                   0
others_willing_to_consume_again   

In [5]:
# 这里每一个标签有 4 个类别，分别是 -2，-1,0,1 ，为了后续分类器的实现，这里将所有标签 + 2，变为 0,1,2,3
columns = ['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']

file[columns] += 2

In [6]:
import re
def remove_special_token(text):
    text = re.sub('\n','',text)
    text = re.sub(r'[_"\-;；%()|+&=*%.。，！？￥,!?:#$@\[\]/]', '', text)
    text = re.sub(r'\'', '', text)
    text = re.sub(r'[a-z]', '', text)
    text = re.sub(r'[A-Z]', '', text)
    text = re.sub(r'[【】《》（）“”’‘：⋯<>『』、‿≖✧▶★⭐️☆˘д❖😄👍😱😋🐮😊]', '', text)
    text = re.sub(r'[\^￣—🔥╭♡~～〜﹏≧▽≦〈〉…☁❤Ő௰⊙]', '', text)
    text = re.sub(r'[\x01\x02\x03\x04\x05\x06\x07\x08]', '', text)
    text = re.sub(r'[0-9①②③④⑤⑥⑦⑧⑨⑩๑•̀ㅂ•́🍺]', '', text)
    text = re.sub(r'\r', '',text)
    text = re.sub(r'\\', '',text)
    text = re.sub(r' ', '',text)
    text = re.sub(r'\t', '',text)
    return text

In [ ]:
def remove_special_words(reviews):
    for i in range(len(reviews)):
        reviews[i] = remove_special_token(reviews[i])

In [7]:
num_lines = len(file.content)
print(num_lines)
# 用 re 来去除标点符号
for i in range(num_lines):
    file.content[i] = remove_special_token(file.content[i])

105000


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [8]:
from transformers import BertTokenizer

pretrained_weights = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [9]:
# 找出所有不在 vocab 中的字和表情
def out_of_vocab(content):
    unk_words = defaultdict(list)

    stics = []
    for i in range(len(content)):
        unk_count = 0
        for word in content[i]:
            if word not in tokenizer.vocab.keys():
                unk_count += 1
                unk_words[i].append(word)
        if unk_count > 0:
            stics += unk_words[i] 
    return set(stics)

In [10]:
my_stop_words = out_of_vocab(file.content)
my_stop_words

{'扞',
 '箅',
 'ㅎ',
 '榅',
 '🎉',
 '┮',
 '😇',
 '✳',
 '蛒',
 '螘',
 '\ue14c',
 '♒',
 '좋',
 '筯',
 '😆',
 '🌸',
 '⊂',
 'ु',
 '͈',
 '¬',
 '湴',
 '🌹',
 'è',
 '📚',
 '鯧',
 '跐',
 '骓',
 '🐏',
 '笊',
 'ฺ',
 '墁',
 '⚾',
 '\ue40a',
 '硌',
 '粯',
 '箜',
 '呶',
 '姽',
 '丷',
 'િ',
 '叄',
 '拶',
 '노',
 '栲',
 '🍎',
 '👉',
 '攰',
 '虼',
 '💢',
 '💅',
 '🚌',
 '镚',
 '🍼',
 '㕑',
 'ᕘ',
 'ᖗ',
 'Ｙ',
 'ө',
 '蜩',
 '🙆',
 '埯',
 '🏻',
 '↝',
 '⃘',
 '鈡',
 '♨',
 '🍆',
 '❵',
 'ˌ',
 '👷',
 '韾',
 'Ｏ',
 '芛',
 '枓',
 '郿',
 '집',
 '₎',
 '˭',
 '旴',
 '꒰',
 '廻',
 '鶗',
 '🈶',
 '濞',
 '🙂',
 '㊙',
 '🤗',
 '\uf702',
 '亱',
 '澥',
 '謽',
 'ᵌ',
 '맛',
 '凃',
 '🐙',
 '蟝',
 '棈',
 '૭',
 '燑',
 'û',
 '璔',
 'ଘ',
 '🏠',
 '糁',
 '🍉',
 '埒',
 '鲺',
 '⑪',
 '樅',
 '੭',
 'べ',
 '💣',
 '稥',
 'ԅ',
 '鱘',
 '💝',
 '℅',
 '湌',
 '鲴',
 '秾',
 '㸆',
 '殄',
 '哙',
 '🆚',
 '吆',
 '՞',
 'ᵕ',
 '蝽',
 '镋',
 '撺',
 '\u200c',
 '↙',
 '\xa0',
 '🤒',
 '㕏',
 '蛄',
 '\ufeff',
 '∑',
 '煵',
 '🍄',
 'ˑ',
 '坻',
 '￤',
 '❥',
 '͓',
 '斕',
 '揿',
 '蚬',
 '∪',
 '姘',
 '戗',
 '😭',
 'ơ',
 '恹',
 '꒳',
 '้',
 '沷',
 '餜',
 '👑',
 '屙',
 '坉',
 '嶝',


In [16]:
# 去除不在 vocab 中的字
for i in range(len(file.content)):
    file.content[i] = ''.join([word for word in file.content[i] if word not in my_stop_words])

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# 查看 评论的长度分布
def creat_text_length(text):
    length = []
    for sentence in text:
        length.append(len(tokenizer.tokenize(sentence)))
    return pd.DataFrame(length, columns=['counts'])

In [12]:
review_length = creat_text_length(file.content)
review_length.describe()

,counts
count,105000.000000
mean,295.182552
std,155.330792
min,11.000000
25%,199.000000
50%,243.000000
75%,331.000000
max,1812.000000


In [15]:
print(np.percentile(review_length.counts, 1))
print('the next:')
print(np.percentile(review_length.counts, 82))
print(np.percentile(review_length.counts, 85))
print(np.percentile(review_length.counts, 90))
print(np.percentile(review_length.counts, 92))
print(np.percentile(review_length.counts, 93))

163.0
the next:
381.0
411.0
472.0
507.0
530.0


In [18]:
file.to_csv('./my_data/sentiment_analysis_trainingset.csv', index=False)

### 同样处理 validation 和 test 评论文本

In [29]:

comment_val = pd.read_csv('./comment-classification/ai_challenger_sentiment_analysis_validationset_20180816/sentiment_analysis_validationset.csv')
comment_test = pd.read_csv('./comment-classification/ai_challenger_sentiment_analysis_testa_20180816/sentiment_analysis_testa.csv') 

In [30]:
comment_val.head(1)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""哎，想当年来佘山的时候，啥都没有，三品香算镇上最大看起来最像样的饭店了。菜品多，有点太多，...",-2,-2,-2,0,-2,-2,-2,1,...,-2,-2,-2,-2,-2,0,-2,-2,1,0


In [31]:
comment_test.head(1)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
print(len(comment_val.id))
comment_val.isnull().sum()

15000


id                                          0
content                                     0
location_traffic_convenience                0
location_distance_from_business_district    0
location_easy_to_find                       0
service_wait_time                           0
service_waiters_attitude                    0
service_parking_convenience                 0
service_serving_speed                       0
price_level                                 0
price_cost_effective                        0
price_discount                              0
environment_decoration                      0
environment_noise                           0
environment_space                           0
environment_cleaness                        0
dish_portion                                0
dish_taste                                  0
dish_look                                   0
dish_recommendation                         0
others_overall_experience                   0
others_willing_to_consume_again   

In [33]:
print(len(comment_test.id))
comment_test.isnull().sum()

15000


id                                              0
content                                         0
location_traffic_convenience                15000
location_distance_from_business_district    15000
location_easy_to_find                       15000
service_wait_time                           15000
service_waiters_attitude                    15000
service_parking_convenience                 15000
service_serving_speed                       15000
price_level                                 15000
price_cost_effective                        15000
price_discount                              15000
environment_decoration                      15000
environment_noise                           15000
environment_space                           15000
environment_cleaness                        15000
dish_portion                                15000
dish_taste                                  15000
dish_look                                   15000
dish_recommendation                         15000


In [34]:
comment_val[columns] += 2

In [26]:
def remove_special_words(reviews):
    for i in range(len(reviews)):
        reviews[i] = remove_special_token(reviews[i])

In [35]:
remove_special_words(comment_val.content)
remove_special_words(comment_test.content)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
val_stop_words = out_of_vocab(comment_val.content)
test_stop_words = out_of_vocab(comment_test.content)

In [37]:
def remove_oov(reviews, my_stop_words):
    for i in range(len(reviews)):
        reviews[i] = ''.join([word for word in reviews[i] if word not in my_stop_words])

In [42]:
remove_oov(comment_val.content, val_stop_words)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
remove_oov(comment_test.content, test_stop_words)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
comment_val.to_csv('./my_data/sentiment_analysis_validationset.csv', index=False)
comment_test.to_csv('./my_data/sentiment_analysis_testset.csv', index=False)

In [46]:
train = pd.read_csv('./my_data/sentiment_analysis_trainingset.csv')
val = pd.read_csv('./my_data/sentiment_analysis_validationset.csv')
test = pd.read_csv('./my_data/sentiment_analysis_testset.csv')

In [47]:
train.head(1)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,吼吼吼萌死人的棒棒糖中了大众点评的霸王餐太可爱了一直就好奇这个棒棒糖是怎么个东西大众点评给了...,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,3,0,3,0


In [51]:
train.content[0]

'吼吼吼萌死人的棒棒糖中了大众点评的霸王餐太可爱了一直就好奇这个棒棒糖是怎么个东西大众点评给了我这个土老冒一个见识的机会看介绍棒棒糖是用德国糖做的不会很甜中间的照片是糯米的能食用真是太高端大气上档次了还可以买蝴蝶结扎口送人可以买礼盒我是先打的卖家电话加了微信给卖家传的照片等了几天卖家就告诉我可以取货了去大官屯那取的虽然连卖家的面都没见到但是还是谢谢卖家送我这么可爱的东西太喜欢了这哪舍得吃啊'

In [48]:
val.head(1)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,哎想当年来佘山的时候啥都没有三品香算镇上最大看起来最像样的饭店了菜品多有点太多感觉啥都有杂都...,0,0,0,2,0,0,0,3,...,0,0,0,0,0,2,0,0,3,2


In [52]:
val.content[0]

'哎想当年来佘山的时候啥都没有三品香算镇上最大看起来最像样的饭店了菜品多有点太多感觉啥都有杂都不足以形容随便点些居然口味什么的都好还可以价钱自然是便宜当震惊元宝虾和椒盐九肚鱼都不错吃不过近来几次么味道明显没以前好了冷餐里面一个凉拌海带丝还可以酸酸甜甜的镇上也有了些别的大点的饭店所以不是每次必来了对了这家的生意一如既往的超级好不定位基本吃不到不过佘山这边的人吃晚饭很早的所以稍微晚点去就很空了'

In [49]:
test.head(1)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,我想说他们家的优惠活动好持久啊我预售的时候买的券前两天心血来潮去吃的活动还在继续首先说下服务...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
test.content[0]

'我想说他们家的优惠活动好持久啊我预售的时候买的券前两天心血来潮去吃的活动还在继续首先说下服务因为和男票开车去的有点不认路老板很耐心的在电话里帮我们指路到了门店之后也帮我们推荐了他们家做的比较地道的伤心凉粉说是厨师是四川那边来的环境呢比较简单干净去的时候下午一点多了还有四五桌人在用餐口味对于我而言点了麻辣的口感正正好男票比较能吃辣相对而言觉得他们家的麻辣口感麻有了辣还欠缺一点老板娘说考虑到客人口味不同所以没敢放太多辣椒能吃辣的朋友可以考虑下单之前和老板先说好鱼呢我们选的是黑鱼斤的鱼加上一盆我以为没有什么东西实际上东西很多的锅底我们吃的饱饱的最后以为吃的差不多了打包一看简直像没动过一样分量还是满足的鱼比较新鲜伤心凉粉很辣不过口味也蛮好吃的总的来说性价比还是可以的两个人吃了大概左右用了团购券的话一百块不到会考虑下次再来'